In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys,os
sys.path.append('/home/heyangle/Desktop/ScanCraft/ScanCraft')
sys.path.append('/home/vooum/Desktop/ScanCraft/ScanCraft')
sys.path.append('C://Users//vooum//Desktop//ScanCraft//ScanCraft')

# collect all sample documents

In [3]:
import pandas,numpy,copy
from command.file_operations.GetDirectories import GetDirectories
from command.file_operations.GetSamples import GetSamples

In [3]:
# get all folders from folder_path, folders' name should contain folder_key
folder_path='./data_no_omg/'
folder_key='data'
folders=GetDirectories(path=folder_path,keyword=folder_key,numbered=True)

In [38]:
# get all spectrum files from subfolder(if not exist, set to '') in each folders
subfolder=''
sample_collect=[
    GetSamples( path=os.path.join( folder,subfolder), patterns=['inp','spectr','omega'],number_position=-1 )
    for folder in folders
]

In [39]:
samples=sum(sample_collect,[])

In [40]:
samples[-1].items()

dict_items([('spectr', '/home/vooum/Desktop/muon_G_2/data_no_omg/data17/spectr.dat.11559'), ('inp', '/home/vooum/Desktop/muon_G_2/data_no_omg/data17/inp.dat.11559'), ('omega', '/home/vooum/Desktop/muon_G_2/data_no_omg/data17/omega.dat.11559')])

In [7]:
len(samples)

47326

# Get all spectrum text

In [9]:
from multiprocessing import Pool
def get_spectr_text(sample):
    spectr_file=sample['spectr']
    with open(spectr_file,'r') as spec:
        text=spec.readlines()
    return text
with Pool(6) as P:
    text_list=P.map(get_spectr_text,samples)

# Read spectrum files

In [34]:
from command.nexus.NMSSMTools import NToolsOutput
spectrum_list=[
    NToolsOutput(text=text) for text in text_list
]
# old, not recommended
# for sample,text in zip(samples,text_list):
#     sample.spectr=NToolsOutput(text=text)

In [35]:
spectrum_list[-1].constraints

['# DM relic density too large',
 '# Muon magn. mom. more than 2 sigma away',
 '# No Higgs in the 122.1-128.1 GeV mass range',
 '# b -> c tau nu more than 2 sigma away (as SM)',
 '# k_bot(H_SM) more than 2 sigma away (as SM)',
 '# k_tau(H_SM) more than 2 sigma away (as SM)']

In [21]:
# all constraints
consts=set()
for spec in spectrum_list:
    consts.update(spec.constraints)
consts

{'# B_bsm(H_SM) more than 2 sigma away',
 '# B_s -> mu+ mu- more than 2 sigma away',
 '# DM direct detection rate too large (SI)',
 '# DM relic density too large',
 '# DM relic density too small',
 '# Landau Pole below MGUT',
 '# Muon magn. mom. more than 2 sigma away',
 '# No Higgs in the 122.1-128.1 GeV mass range',
 '# b -> c tau nu more than 2 sigma away (as SM)',
 '# b -> s gamma more than 2 sigma away',
 '# k_bot(H_SM) more than 2 sigma away (as SM)',
 '# k_tau(H_SM) more than 2 sigma away (as SM)'}

In [22]:
ignores=['Landau Pole'#27
        ,'DM direct detection'
        ,'relic density'
#         ,'Muon magn'#37
        ,'b -> c tau nu'#58 always keep alive
        ,'k_bot(H_SM)'
        ,'k_tau(H_SM)'
        ,'B_bsm(H_SM)'
        ]

In [23]:
def data_forX2(spectr):
    mN=spectr('mass',1000022)
    xz=spectr('REDCOUP',(1,5))
    xw=xz
    xb=spectr('REDCOUP',(1,3))
    xt=spectr('REDCOUP',(1,1))
    xtau=spectr('REDCOUP',(1,4))
    xg=spectr('REDCOUP',(1,7))
    xj=spectr('REDCOUP',(1,6))
    return{
        'mh':spectr('mass',25)
        ,'bsg':spectr('LOWEN',1)
        ,'bmu':spectr('LOWEN',4)
        ,'relic':spectr('LOWEN',10)
        ,'Psi':(mN,spectr('LOWEN',20))
        ,'Nsd':(mN,spectr('LOWEN',30))
        ,'Psd':(mN,spectr('LOWEN',40))
        ,'Higgs_coupling':(xz,xw,xb,xt,xtau,xg,xj)
    }

In [25]:
# Chisquare
from command.Experiments.Chi_square_Test import Chisq
X2_list=[Chisq(**data_forX2(spec)) for spec in spectrum_list]

In [26]:
X2_list[-1]

{'mh': 6.054038309255542,
 'bsg': 0.05805461983822662,
 'bmu': 2.758435843230604,
 'relic': 1.0186620016551746,
 'Psi': 0.0,
 'Nsd': 0.0,
 'Psd': 0.0,
 'Higgs_coupling': 2.620038875129047}

# filter

In [27]:
def good(spectr,X2):
    spectr.ignore=ignores
    if spectr.error : return False
    if spectr.constraints : return False
    if X2['Higgs_coupling']>14.067: return False
    return True

In [42]:
good_samples=[[sample,spec,chi2] for sample,spec,chi2 in zip(samples,spectrum_list,X2_list) if good(spec,chi2)]
len(good_samples)

2535

In [43]:
good_samples[-1]

[{'spectr': '/home/vooum/Desktop/muon_G_2/data_no_omg/data17/spectr.dat.11544',
  'omega': '/home/vooum/Desktop/muon_G_2/data_no_omg/data17/omega.dat.11544',
  'inp': '/home/vooum/Desktop/muon_G_2/data_no_omg/data17/inp.dat.11544'},
 {'mh': 0.2124261679063072,
  'bsg': 1.9585604749387735,
  'bmu': 1.4540110969376312,
  'relic': 1.3641173372155373,
  'Psi': 0.0,
  'Nsd': 0.0,
  'Psd': 0.0,
  'Higgs_coupling': 2.4915196364414207}]

In [45]:
number=0
path='./good_no_omg/'
for sample,spec,chi2 in good_samples:
    dest={'inp':os.path.join(path,f'inp.dat.{number}'),
          'spectr':os.path.join(path,f'spectr.dat.{number}'),
          'omega':os.path.join(path,f'omega.dat.{number}')
         }
    sample.CopyTo(dest)
    number+=1

# re-collect good samples

In [4]:
from command.file_operations.GetSamples import GetSamples
good_samples=GetSamples('./good_no_omg/',patterns=['inp','spectr','omega'])
good_samples[-1]

{'omega': '/home/vooum/Desktop/muon_G_2/good_no_omg/omega.dat.2534',
 'spectr': '/home/vooum/Desktop/muon_G_2/good_no_omg/spectr.dat.2534',
 'inp': '/home/vooum/Desktop/muon_G_2/good_no_omg/inp.dat.2534'}

# rerun

In [5]:
from command.scan.scan import scan
mold=scan()

In [6]:
mold.AddScalar('tanB','MINPAR',3,1.,60.) 
mold.AddScalar('M1','EXTPAR',1  ,20.    ,2000.) 
mold.AddScalar('M2','EXTPAR',2  ,100.    ,2000.)
mold.AddScalar('Atop','EXTPAR',11  ,  -5e3    ,5e3)
mold.AddFollower('Abottom','EXTPAR'   ,12,'Atop')
mold.AddScalar('Atau','EXTPAR'   ,13  ,  100.      ,2000.)
mold.AddScalar('Amuon','EXTPAR'   ,16  ,  100.      ,2000.)
mold.AddScalar('MtauL','EXTPAR'   ,33,100.,2000.)
mold.AddScalar('MmuonL','EXTPAR'   ,32,100.,2000.)
mold.AddScalar('MtauR','EXTPAR'   ,36,100.,2000.)
mold.AddScalar('MmuonR','EXTPAR'   ,35,100.,2000.)
# mold.AddScalar('MQ3L','EXTPAR'   ,43,	100.,	2.e3)#
# mold.AddScalar('MtopR'	,'EXTPAR'   ,46,	100.,	2.e3)#
# mold.AddFollower('MbottomR','EXTPAR'  ,49,'MtopR')#
mold.AddScalar('Lambda','EXTPAR'  ,61  ,1e-3    ,.75 ,prior_distribution='exponential')
mold.AddScalar('Kappa','EXTPAR'   ,62 ,-0.75    ,.75 ,prior_distribution='exponential')
# mold.AddScalar('A_Lambda','EXTPAR' ,63,-3.e3,3.e3)
mold.AddScalar('A_kappa','EXTPAR' ,64,-2.e3,2.e3)
mold.AddScalar('mu_eff','EXTPAR'  ,65,100.,2000.)

In [23]:
from queue import SimpleQueue
inp_queue=SimpleQueue()

In [24]:
from copy import deepcopy
for sample in good_samples[:10]:
    new_inp=deepcopy(mold)
    new_inp.GetValue(sample['inp'])
    inp_queue.put(new_inp)
inp_queue.qsize()

10

In [31]:
from command.parallel.MT_NTools import MT_NTools
MP=MT_NTools(6)

old folder ./output has renamed to ./output_20200314_224955


In [32]:
resutls=MP.Run(inp_queue) # results=MP.all_points

Calculations begin at Sat Mar 14 22:50:32 2020
  Threads:	6
  points:	10
All points done. Use {work_time}
all data files stored in 'all_points' attribute


[{'inp': 'pylon/NMSSMTools_5.5.2_prob_0/record/inp.0',
  'spectr': 'pylon/NMSSMTools_5.5.2_prob_0/record/spectr.0',
  'omega': 'pylon/NMSSMTools_5.5.2_prob_0/record/omega.0'},
 {'inp': 'pylon/NMSSMTools_5.5.2_prob_0/record/inp.1',
  'spectr': 'pylon/NMSSMTools_5.5.2_prob_0/record/spectr.1',
  'omega': 'pylon/NMSSMTools_5.5.2_prob_0/record/omega.1'},
 {'inp': 'pylon/NMSSMTools_5.5.2_prob_1/record/inp.0',
  'spectr': 'pylon/NMSSMTools_5.5.2_prob_1/record/spectr.0',
  'omega': 'pylon/NMSSMTools_5.5.2_prob_1/record/omega.0'},
 {'inp': 'pylon/NMSSMTools_5.5.2_prob_1/record/inp.1',
  'spectr': 'pylon/NMSSMTools_5.5.2_prob_1/record/spectr.1',
  'omega': 'pylon/NMSSMTools_5.5.2_prob_1/record/omega.1'},
 {'inp': 'pylon/NMSSMTools_5.5.2_prob_2/record/inp.0',
  'spectr': 'pylon/NMSSMTools_5.5.2_prob_2/record/spectr.0',
  'omega': 'pylon/NMSSMTools_5.5.2_prob_2/record/omega.0'},
 {'inp': 'pylon/NMSSMTools_5.5.2_prob_3/record/inp.0',
  'spectr': 'pylon/NMSSMTools_5.5.2_prob_3/record/spectr.0',
  'o

In [35]:
MP.DeleteData()